### Installation
Install the packages required for executing this notebook.

In [ ]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Check the versions of the packages you installed. The KFP SDK version should be >2.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.14.4
google-cloud-aiplatform==1.118.0
google_cloud_pipeline_components version: 2.21.0


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [ ]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "de2025-472319"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2025_2062061"

#### Pipeline Component : Data Ingestion

In [4]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')

## Splitting data into train and test


In [ ]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn==1.3.2"],
    base_image="python:3.10.7-slim"
)
def train_test_split(dataset: Input[Dataset], dataset_train: Output[Dataset], dataset_test: Output[Dataset]):
    '''train_test_split'''
    import pandas as pd
    import logging 
    import sys
    from sklearn.model_selection import train_test_split
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
    # Split into train and test
    data = pd.read_csv(dataset.path+".csv", index_col=None)
    train, test = train_test_split(data, test_size=0.2, random_state=42)
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Training-XGBoost


In [ ]:
@dsl.component(
    packages_to_install=['pandas', 'xgboost', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_xgboost (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a XGBoost model with default parameters'''
    import pandas as pd
    import numpy as np
    import pickle
    from xgboost import XGBRegressor
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import json
    import logging 
    import sys
    import os
        
    # Load the train and test sets into dataframes
    df_train = pd.read_csv(train_set.path+".csv")
    df_test = pd.read_csv(test_set.path+".csv")

    logging.info(df_train.columns)
    logging.info(df_test.columns)  
        
    # split into input (X) and output (Y) variables
    X_train, y_train = df_train.drop('house_value', axis=1), df_train['house_value']
    X_test, y_test = df_test.drop('house_value', axis=1), df_test['house_value']

    # Train XGBoost Model to get feature importance
    xgb_model = XGBRegressor(n_estimators=100, random_state=42)
    xgb_model.fit(X_train, y_train)
    xgb_importances = xgb_model.feature_importances_
    xgb_indices = np.argsort(xgb_importances)[::-1][:15]  # Top 15 features for XGBoost
    top_15_features_xgb = X_train.columns[xgb_indices]

    # Train-Test Split with Top 15 Features
    X_train = X_train[top_15_features_xgb]
    X_test = X_test[top_15_features_xgb]

    # Grid Search for XGBoost with Top 15 Features
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'subsample': [0.8, 1]
    }

    # Grid Search
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    # Best XGBoost model after grid search
    best_xgb_model = grid_search.best_estimator_

    # XGBoost with top 15 features
    y_pred = best_xgb_model.predict(X_test)

    # evaluate the model
    y_pred = best_xgb_model.predict(X_test)

    # Calculate evaluation metrics and store them in a dictionary
    metrics_dict = {
        "mse": mean_squared_error(y_test, y_pred),
        "mae": mean_absolute_error(y_test, y_pred),
        "r2": r2_score(y_test, y_pred),
        "rmse": np.sqrt(mean_squared_error(y_test, y_pred))
    }    
    logging.info(metrics_dict)  
    
    # Store some metadata
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algorithm"] = "xgboost"
    
    # Save the model to a pickle file
    model_file = out_model.path + ".pkl"
    with open(model_file, 'wb') as f:  
        pickle.dump(best_xgb_model, f)   
    
    # Return the metrics dictionary as an output
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Training LinearRegression

In [ ]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_lr (features: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    '''train a LinearRegression with default parameters'''
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import json
    import logging 
    import sys
    import os
    import pickle  
       
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Load the train and test sets into dataframes
    df_train = pd.read_csv(train_set.path+".csv")
    df_test = pd.read_csv(test_set.path+".csv")

    logging.info(df_train.columns)
    logging.info(df_test.columns)  
        
    # split into input (X) and output (Y) variables
    X_train, y_train = df_train.drop('house_value', axis=1), df_train['house_value']
    X_test, y_test = df_test.drop('house_value', axis=1), df_test['house_value']

    # Fit the model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_importances = np.abs(lr_model.coef_)  # Absolute value of coefficients
    lr_indices = np.argsort(lr_importances)[::-1][:15]  # Top 15 features for Linear Regression
    top_15_features_lr = X_train.columns[lr_indices]
    
    # Train-Test Split with Top 15 Features
    X_train = X_train[top_15_features_lr]
    X_test = X_test[top_15_features_lr]

    # Retrain model using only top 15 features
    lr_model_top_15 = LinearRegression()
    lr_model_top_15.fit(X_train, y_train)

    # Predict using the retrained model
    y_pred = lr_model_top_15.predict(X_test)

    # Calculate evaluation metrics and store them in a dictionary
    metrics_dict = {
        "mse": mean_squared_error(y_test, y_pred),
        "mae": mean_absolute_error(y_test, y_pred),
        "r2": r2_score(y_test, y_pred),
        "rmse": np.sqrt(mean_squared_error(y_test, y_pred))
    }    
    logging.info(metrics_dict)  
    
    # Store some metadata
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algorithm"] = "lr"
    
    # Save the model to a pickle file
    model_file = out_model.path + ".pkl"
    with open(model_file, 'wb') as f:  
        pickle.dump(lr_model_top_15, f)   
    
    # Return the metrics dictionary as an output
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Algorithm Selection 

In [ ]:
@dsl.component(
    base_image="python:3.10.7-slim"
)
def compare_model(xgboost_metrics: dict, lr_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(xgboost_metrics)
    logging.info(lr_metrics)
    if xgboost_metrics.get("r2") > lr_metrics.get("r2"):
        return "XGBoost"
    else :
        return "LR"

### Upload Model and Metrics to Google Bucket 

In [10]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    blob = bucket.blob(str(model.metadata["algo"]) + '_model' + str(model.metadata["file_type"])) 
    blob.upload_from_filename(model.path + str(model.metadata["file_type"]))       
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Define the Pipeline

In [11]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="diabetes-prdictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str, testset_filename: str):
    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_mlp_job_run_op = train_mlp(
        features=di_op.outputs["dataset"]
    )
    
     
    training_lr_job_run_op = train_lr(
        features=di_op.outputs["dataset"]
    )
    
    pre_di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    ).after(training_mlp_job_run_op, training_lr_job_run_op)
        
        
    comp_model__op = compare_model(mlp_metrics=training_mlp_job_run_op.outputs["metrics"],
                                       lr_metrics=training_lr_job_run_op.outputs["metrics"]).after(training_mlp_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.If(comp_model__op.output=="MLP"):
        predict_mlp_job_run_op = predict_mlp(
            model=training_mlp_job_run_op.outputs["out_model"],      
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_mlp_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_mlp_job_run_op.outputs['out_model']
        ).after(predict_mlp_job_run_op)
        
    with dsl.If(comp_model__op.output=="LR"):
        predict_lr_job_run_op = predict_lr(
            model=training_lr_job_run_op.outputs["out_model"],     
            features=pre_di_op.outputs["dataset"]
        )
        upload_model_lr_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_lr_job_run_op.outputs['out_model']
        ).after(predict_lr_job_run_op)  

#### Compile the pipeline into a JSON file

In [12]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='diabetes_predictor_training_pipeline.yaml')

#### Submit the pipeline run

In [14]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    staging_bucket=PIPELINE_ROOT,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="diabetes-predictor-ct-pipeline",
    enable_caching=False,
    template_path="diabetes_predictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2025',  # makesure to use your data bucket name 
        'trainset_filename': 'training_set.csv',     # makesure to upload these to your data bucket from DE2024/lab4/data
        'testset_filename': 'prediction_set.csv',    # makesure to upload these to your data bucket from DE2024/lab4/data
        'model_repo':'models_de2025' # makesure to use your model bucket name 
    }
)

job.run()